In [ ]:

Does not make sense
Cannot be finetuned with all 4 losses. The term 1 and term 4 are contradictory.

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda:5')
else:
    device = torch.device('cpu')
    
print(device)
device = torch.device('cpu')
cpu
# the deep neural network
class MLP(torch.nn.Module):
    def __init__(self, layers, activation="relu", init="xavier"):
        super(MLP, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        if activation == "relu":
            self.activation = torch.nn.ReLU()
        elif activation == "tanh":
            self.activation = torch.nn.Tanh()
        elif activation == "gelu":
            self.activation = torch.nn.GELU()
        else:
            raise ValueError("Unspecified activation type")
        
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)

        if init=="xavier":
            self.xavier_init_weights()
        elif init=="kaiming":
            self.kaiming_init_weights()
    
    def xavier_init_weights(self):
        with torch.no_grad():
            print("Initializing Network with Xavier Initialization..")
            for m in self.layers.modules():
                if hasattr(m, 'weight'):
                    nn.init.xavier_uniform_(m.weight)
                    m.bias.data.fill_(0.0)

    def kaiming_init_weights(self):
        with torch.no_grad():
            print("Initializing Network with Kaiming Initialization..")
            for m in self.layers.modules():
                if hasattr(m, 'weight'):
                    nn.init.kaiming_uniform_(m.weight)
                    m.bias.data.fill_(0.0)
                        
    def forward(self, x):
        out = self.layers(x)
        return out
    
class DataGenerator(torch.utils.data.Dataset):
    def __init__(self, X):
        self.X = X
        
    def __getitem__(self, index):
        return self.X[index]
    
    def __len__(self):
        return len(self.X)
data_df = pd.read_csv("all_data_lake_modeling_in_time_wHeat.csv")
time = data_df['time']
data_df = data_df.drop(columns=['time'])
data_df
display(data_df.head)
<bound method NDFrame.head of         depth  AirTemp_degC  Longwave_Wm-2  Latent_Wm-2  Sensible_Wm-2  \
0           1     10.715021     695.937161   -56.268384     -23.291619   
1           2     10.715021     695.937161   -56.268384     -23.291619   
2           3     10.715021     695.937161   -56.268384     -23.291619   
3           4     10.715021     695.937161   -56.268384     -23.291619   
4           5     10.715021     695.937161   -56.268384     -23.291619   
...       ...           ...            ...          ...            ...   
141570     21     21.695001     832.055902   -93.493728     -12.091248   
141571     22     21.695001     832.055902   -93.493728     -12.091248   
141572     23     21.695001     832.055902   -93.493728     -12.091248   
141573     24     21.695001     832.055902   -93.493728     -12.091248   
141574     25     21.695001     832.055902   -93.493728     -12.091248   

        Shortwave_Wm-2  lightExtinct_m-1  ShearVelocity_mS-1  \
0                  0.0          0.255324            1.085448   
1                  0.0          0.255324            1.085448   
2                  0.0          0.255324            1.085448   
3                  0.0          0.255324            1.085448   
4                  0.0          0.255324            1.085448   
...                ...               ...                 ...   
141570             0.0          1.162606            1.049822   
141571             0.0          1.162606            1.049822   
141572             0.0          1.162606            1.049822   
141573             0.0          1.162606            1.049822   
141574             0.0          1.162606            1.049822   

        ShearStress_Nm-2     Area_m2  ...  buoyancy  diffusivity  temp_heat00  \
0               0.002290  36000000.0  ... -0.000006     0.000037    16.283408   
1               0.002290  36000000.0  ... -0.000006     0.000037    16.283408   
2               0.002290  36000000.0  ... -0.000007     0.000037    16.287068   
3               0.002290  36000000.0  ...  0.000021     0.000037    16.291257   
4               0.002290  36000000.0  ...  0.000180     0.000025    16.278268   
...                  ...         ...  ...       ...          ...          ...   
141570          0.002174  36000000.0  ...  0.000637     0.000014     8.537687   
141571          0.002174  36000000.0  ...  0.000474     0.000016     7.466041   
141572          0.002174  36000000.0  ...  0.000307     0.000020     6.396040   
141573          0.002174  36000000.0  ...  0.000134     0.000028     5.326140   
141574          0.002174  36000000.0  ...  0.000134     0.000028     4.256585   

        temp_diff01  day_of_year  time_of_day  temp_mix02  temp_conv03  \
0         16.149422          155            1   16.206994    16.241692   
1         16.267964          155            1   16.206994    16.247184   
2         16.286071          155            1   16.286071    16.251763   
3         16.288695          155            1   16.288695    16.251763   
4         16.270156          155            1   16.270156    16.256606   
...             ...          ...          ...         ...          ...   
141570     8.537921          213           23    8.537921     8.537921   
141571     7.466186          213           23    7.466186     7.466186   
141572     6.396125          213           23    6.396125     6.396125   
141573     5.326179          213           23    5.326179     5.326179   
141574     4.256585          213           23    4.256585     4.256585   

        obs_temp  input_obs  
0         16.409     16.350  
1         16.480     16.426  
2         16.130     16.088  
3         15.827     15.789  
4         16.270     16.240  
...          ...        ...  
141570    11.673     11.673  
141571    11.673     11.673  
141572    11.673     11.673  
141573    11.673     11.673  
141574    11.673     11.673  

[141575 rows x 22 columns]>
training_frac = 0.60
depth_steps = 25
number_days = len(data_df)//depth_steps
n_obs = int(number_days*training_frac)*depth_steps
print(f"Number of days total: {number_days}")
print(f"Number of training points: {n_obs}")
Number of days total: 5663
Number of training points: 84925
Normalizing Data
data = data_df.values

train_data = data[:n_obs]
test_data = data[n_obs:]

train_time = time[:n_obs]
test_time = time[n_obs:]

#performing normalization on all the columns
scaler = StandardScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
m0_output_column_ix = [data_df.columns.get_loc(column) for column in ['temp_heat00']]
Loading Models
m0_input_columns = ['depth', 'AirTemp_degC', 'Longwave_Wm-2', 'Latent_Wm-2', 'Sensible_Wm-2', 'Shortwave_Wm-2',
                'lightExtinct_m-1','Area_m2', 
                 'day_of_year', 'time_of_day', 'input_temp']
m0_output_columns = ['temp_heat00']

m0_input_column_ix = [data_df.columns.get_loc(column) for column in m0_input_columns]
m0_output_column_ix = [data_df.columns.get_loc(column) for column in m0_output_columns]

m0_PATH =  f"./saved_models/heating_model_time.pth"
m0_layers = [len(m0_input_columns), 32, 32, len(m0_output_columns)]

heating_model = MLP(m0_layers, activation="gelu")
m0_checkpoint = torch.load(m0_PATH, map_location=torch.device('cpu'))
heating_model.load_state_dict(m0_checkpoint)
heating_model = heating_model.to(device)
Initializing Network with Xavier Initialization..
Heat Diffusion Model
m1_input_columns = ['depth', 'AirTemp_degC', 'Longwave_Wm-2', 'Latent_Wm-2', 'Sensible_Wm-2', 'Shortwave_Wm-2',
                'lightExtinct_m-1', 'ShearVelocity_mS-1', 'ShearStress_Nm-2', 'Area_m2', 
                 'buoyancy', 'day_of_year', 'time_of_day', 'diffusivity', 'temp_heat00']
m1_output_columns = ['temp_diff01']

m1_input_column_ix = [data_df.columns.get_loc(column) for column in m1_input_columns]
m1_output_column_ix = [data_df.columns.get_loc(column) for column in m1_output_columns]

m1_PATH = f"./saved_models/heat_diffusion_model_time.pth"
m1_layers = [len(m1_input_columns), 32, 32, len(m1_output_columns)]

heat_diff_model = MLP(m1_layers, activation="gelu")
m1_checkpoint = torch.load(m1_PATH, map_location=torch.device('cpu'))
heat_diff_model.load_state_dict(m1_checkpoint)
heat_diff_model = heat_diff_model.to(device)
Initializing Network with Xavier Initialization..
Mixing Model
m2_input_columns = ['depth', 'ShearVelocity_mS-1', 'ShearStress_Nm-2', 'day_of_year', 'time_of_day', 'temp_diff01']
m2_output_columns = ['temp_mix02']

m2_input_column_ix = [data_df.columns.get_loc(column) for column in m2_input_columns]
m2_output_column_ix = [data_df.columns.get_loc(column) for column in m2_output_columns]

m2_PATH = f"./saved_models/mixing_model_time.pth"
m2_layers = [len(m2_input_columns), 32, 32, len(m2_output_columns)]

mixing_model = MLP(m2_layers, activation="gelu")
m2_checkpoint = torch.load(m2_PATH, map_location=torch.device('cpu'))
mixing_model.load_state_dict(m2_checkpoint)
mixing_model = mixing_model.to(device)
Initializing Network with Xavier Initialization..
Convection Model
m3_input_columns = ['depth', 'day_of_year', 'time_of_day', 'temp_mix02']
m3_output_columns = ['temp_conv03']

m3_input_column_ix = [data_df.columns.get_loc(column) for column in m3_input_columns]
m3_output_column_ix = [data_df.columns.get_loc(column) for column in m3_output_columns]

m3_PATH = f"./saved_models/convection_model_time.pth"
m3_layers = [len(m3_input_columns), 32, 32, len(m3_output_columns)]

convection_model = MLP(m3_layers, activation="gelu")
m3_checkpoint = torch.load(m3_PATH, map_location=torch.device('cpu'))
convection_model.load_state_dict(m3_checkpoint)
convection_model = convection_model.to(device)
Initializing Network with Xavier Initialization..
Ice Model
m4_input_columns = ['depth', 'day_of_year', 'time_of_day', 'temp_conv03']
m4_output_columns = ['temp_total04']

m4_input_column_ix = [data_df.columns.get_loc(column) for column in m4_input_columns]
m4_output_column_ix = [data_df.columns.get_loc(column) for column in m4_output_columns]

m4_PATH = f"./saved_models/ice_model_time.pth"
m4_layers = [len(m4_input_columns), 32, 32, len(m4_output_columns)]

ice_model = MLP(m4_layers, activation="gelu")
m4_checkpoint = torch.load(m4_PATH, map_location=torch.device('cpu'))
ice_model.load_state_dict(m4_checkpoint)
ice_model = ice_model.to(device)
Initializing Network with Xavier Initialization..
obs_temp_columns = ['obs_temp']

obs_temp_columns_ix = [data_df.columns.get_loc(column) for column in obs_temp_columns]
# depth_steps = 24
# train_data = np.reshape(train_data, (train_data.shape[0]//depth_steps, depth_steps, train_data.shape[1]))
# test_data = np.reshape(test_data, (test_data.shape[0]//depth_steps, depth_steps, test_data.shape[1]))
#keeping track of the mean and standard deviations
train_mean = scaler.mean_
train_std = scaler.scale_
# Create data set
batch_size = 1000

assert batch_size % 25 ==0, "Batchsize has to be multiple of 25" 

train_dataset = DataGenerator(train_data)
test_dataset = DataGenerator(test_data)
# train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
# test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                          shuffle=False)
print(train_data.shape)
print(test_data.shape)
(84925, 22)
(56650, 22)
def rmse(true, pred):
    return (((true-pred)**2).mean()**0.5).detach().cpu().numpy()

def l2_error(true, pred):
    return np.linalg.norm(pred.detach().cpu().numpy() - true.detach().cpu().numpy()) / np.linalg.norm(true.detach().cpu().numpy()) 
#keeping track of the mean and standard deviations

input_mean, input_std = train_mean[m1_input_column_ix], train_std[m1_input_column_ix]
output_mean, output_std = train_mean[m1_output_column_ix], train_std[m1_output_column_ix]

mean_diff = torch.tensor(input_mean[m1_input_column_ix[13]]).to(device)
std_diff = torch.tensor(input_std[m1_input_column_ix[13]]).to(device)

mean_temp = torch.tensor(input_mean[m1_input_column_ix[14]]).to(device)
std_temp = torch.tensor(input_std[m1_input_column_ix[14]]).to(device)

mean_out = torch.tensor(output_mean).to(device)
std_out = torch.tensor(output_std).to(device)
    
def implicit_diffusion(diff, temp, dt=3600, dx=1, depth_steps=25):
    # de-normalise data
    diff = diff * std_diff + mean_diff
    diff = diff.view(-1, depth_steps)
    
    # INPUT DATA FROM PREVIOUS MODULE
    t = temp * std_temp + mean_temp # temperature profile from previous module output
    t = t.view(-1, depth_steps)
    
    # IMPLEMENTATION OF CRANK-NICHOLSON SCHEME
#     len_t = t.shape[1]
    y = torch.zeros((t.shape[0], depth_steps, depth_steps), dtype=torch.float64).to(device)

    alpha = (dt/dx**2) * diff

    az = - alpha # subdiagonal
    bz = 2 * (1 + alpha) # diagonal
    cz = - alpha # superdiagonal
    
    bz[:, 0] = 1
    az[:, depth_steps-2] = 0
    bz[:, depth_steps-1] = 1
    cz[:, 0] = 0
    
    az = az[:,1:]
    cz = cz[:,:-1]

    y = torch.diag_embed(bz, offset=0)+torch.diag_embed(az,offset=-1)+torch.diag_embed(cz,offset=1) #slightly efficient way of computing the diagonal matrices
    y[:, depth_steps-1, depth_steps-1] = 1
    
    mn = torch.zeros_like(t)  
    mn[:, 0] = t[:, 0]
    mn[:,depth_steps-1] = t[:, depth_steps-1]
    
    mn[:, 1:depth_steps-1] = alpha[:, 1:depth_steps-1]*t[:, :depth_steps-2] + 2 * (1 - alpha[:,1:depth_steps-1])*t[:,1:depth_steps-1] + alpha[:,1:depth_steps-1]*t[:,1:depth_steps-1] #is be same as the loop
    
    # DERIVED TEMPERATURE OUTPUT FOR NEXT MODULE
    proj = torch.linalg.solve(y, mn)

    mean, std, var = torch.mean(proj), torch.std(proj), torch.var(proj)
    proj = (proj-mean_out)/std_out

    proj = proj.to(torch.float32)
    proj = proj.view(-1, 1)
    return proj
def compute_metrics(model, loader, input_columns, output_columns, train_mean, train_std):
    model.eval()
    y_ = []
    pred_ = []
    mean = torch.tensor(train_mean[output_columns]).to(device)
    std = torch.tensor(train_std[output_columns]).to(device)
    
    for x in iter(loader):
        inputs, target = x[:, input_columns].to(device).float(), x[:, output_columns].to(device).float()
        pred = model(inputs)
        target = target * std + mean
        pred = pred * std + mean
        y_.append(target)
        pred_.append(pred)
    y_ = torch.cat(y_, dim=0) 
    pred_ = torch.cat(pred_, dim=0)
    
    if y_.shape[1]==2:
        rmse_temp = rmse(y_[:,1], pred_[:,1])
        l2_error_temp = l2_error(y_[:,1], pred_[:,1])
    else:
        rmse_temp = rmse(y_[:,0], pred_[:,0])
        l2_error_temp = l2_error(y_[:,0], pred_[:,0])
        
    return rmse_temp, l2_error_temp
print(m1_input_column_ix)
print(m1_input_column_ix[:-1])
print(m1_input_column_ix[:-2])
print(m1_input_column_ix[14])
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 16, 17, 13, 14]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 16, 17, 13]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 16, 17]
14
def get_rollout_predictions(heating_model, heat_diff_model, mixing_model, convection_model, ice_model, loader, plot = True):    
    heating_model.eval()
    heat_diff_model.eval()
    mixing_model.eval()
    convection_model.eval()
    ice_model.eval()

    mean = torch.tensor(train_mean[m4_output_column_ix]).float().to(device)
    std = torch.tensor(train_std[m4_output_column_ix]).float().to(device)

    m1_mean = torch.tensor(train_mean[m1_input_column_ix[-1]]).float().to(device)
    m1_std = torch.tensor(train_std[m1_input_column_ix[-1]]).float().to(device)

#     depthwise_y_pred = []
#     depthwise_y_true = []
    y_ = []
    y_obs_ = []
    pred_ = []
        
    rmse_models = np.zeros((len(loader), 5))
    for ix, x in enumerate(iter(loader)):
        x = x.to(device).float()
        
        m0_input = x[:, m0_input_column_ix]
            
        #model 0
        m0_pred = heating_model(m0_input) #predicts diff and temp
            
        if plot:
            m0_y_true = x[:, m0_output_column_ix[0]] * torch.tensor(train_std[m0_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m0_output_column_ix[0]]).to(device)
            m0_y_pred = m0_pred * torch.tensor(train_std[m0_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m0_output_column_ix[0]]).to(device)
#             print(m1_y_true.shape, m1_y_pred.shape)
#             print("True",m1_y_true)
#             print("Pred",m1_y_pred)
            rmse_models[ix, 0] = rmse(m0_y_true.squeeze(), m0_y_pred.squeeze())
#             print(x[:, m1_output_column_ix[1]])
#             print(m1_pred_temp)
#             print(criterion(m1_pred_temp, x[:, m1_output_column_ix[1]]))
            print("RMSE of after m0", rmse(m0_y_true.flatten(), m0_y_pred.flatten()))

        #model 2
        m1_input = torch.cat([x[:, m1_input_column_ix[:-1]], m0_pred], dim=-1)
        #m1_input = x[:, m1_input_column_ix]
            
        #model 1
        # m1_pred = heat_diff_model(m1_input) #predicts diff and temp
        
        temp_input = m0_y_pred = m0_pred * torch.tensor(train_std[m0_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m0_output_column_ix[0]]).to(device)
       # print(temp_input)
        
        proj = heat_diff_model(m1_input)
        
        pred = implicit_diffusion(proj, m0_pred)        
        m1_pred = pred.to(dtype=torch.float32)
        
        #print(m1_pred)
        
        if plot:
            m1_y_true = x[:, m1_output_column_ix[0]] * torch.tensor(train_std[m1_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m1_output_column_ix[0]]).to(device)
            m1_y_pred = m1_pred * torch.tensor(train_std[m1_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m1_output_column_ix[0]]).to(device)
#             print(m1_y_true.shape, m1_y_pred.shape)
#             print("True",m1_y_true)
#             print("Pred",m1_y_pred)
            rmse_models[ix, 1] = rmse(m1_y_true.squeeze(), m1_y_pred.squeeze())
#             print(x[:, m1_output_column_ix[1]])
#             print(m1_pred_temp)
#             print(criterion(m1_pred_temp, x[:, m1_output_column_ix[1]]))
            print("RMSE of after m1", rmse(m1_y_true.flatten(), m1_y_pred.flatten()))

        #model 2
        m2_input = torch.cat([x[:, m2_input_column_ix[:-1]], m1_pred], dim=-1)
        m2_pred = mixing_model(m2_input)
            
        if plot:
            m2_y_true = x[:, m2_output_column_ix] * torch.tensor(train_std[m2_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m2_output_column_ix[0]]).to(device)
            m2_y_pred = m2_pred * torch.tensor(train_std[m2_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m2_output_column_ix[0]]).to(device)
            rmse_models[ix, 2] = rmse(m2_y_true.squeeze(), m2_y_pred.squeeze())
            print("RMSE of after m2", rmse(m2_y_true.flatten(), m2_y_pred.flatten()))

        #model 3
        m3_input = torch.cat([x[:, m3_input_column_ix[:-1]], m2_pred], dim=-1)
        m3_pred = convection_model(m3_input)
            
        if plot:
            m3_y_true = x[:, m3_output_column_ix] * torch.tensor(train_std[m3_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m3_output_column_ix[0]]).to(device)
            m3_y_pred = m3_pred * torch.tensor(train_std[m3_output_column_ix[0]]).to(device) + torch.tensor(train_mean[m3_output_column_ix[0]]).to(device)
            rmse_models[ix, 3] = rmse(m3_y_true.squeeze(), m3_y_pred.squeeze())
            print("RMSE of after m3", rmse(m3_y_true.flatten(), m3_y_pred.flatten()))

        #model 4
        m4_input = torch.cat([x[:, m4_input_column_ix[:-1]], m3_pred], dim=-1)
        m4_pred = ice_model(m4_input)
            
        if plot:
            m4_y_true = x[:, m4_output_column_ix] * std + mean
            m4_y_pred = m4_pred * std + mean
            rmse_models[ix, 4] = rmse(m4_y_true.squeeze(), m4_y_pred.squeeze())
            print("RMSE of after m4", rmse(m4_y_true.flatten(), m4_y_pred.flatten()))

        y_true = x[:, m4_output_column_ix] * std + mean
        y_obs = x[:, obs_temp_columns_ix] * train_std[obs_temp_columns_ix[0]] + train_mean[obs_temp_columns_ix[0]]
        pred = m4_pred * std + mean
        
        y_.append(y_true)
        y_obs_.append(y_obs)
        pred_.append(pred)

    y_ = torch.cat(y_, dim=0)
    y_obs_ = torch.cat(y_obs_, dim=0)
    pred_ = torch.cat(pred_, dim=0) 
    
    #if plot:   
     #        rmse_models = rmse_models.mean(axis=0)
     #        plt.figure(figsize=(12,8))
     #        plt.plot(rmse_models[:, 0], label="RMSE after Heating Model")
     #        plt.plot(rmse_models[:, 1], label="RMSE after Heat-Diffusion Model")
     #        plt.plot(rmse_models[:, 2], label="RMSE after Mixing Model")
     #        plt.plot(rmse_models[:, 3], label="RMSE after Convection Model")
     #        plt.plot(rmse_models[:, 4], label="RMSE after Ice Model")
     #        plt.legend(loc="upper left", fontsize=12)
     #        plt.xlabel("Depth", fontsize=12)
     #        plt.ylabel("RMSE", fontsize=12)
     #        plt.grid("on", alpha=0.5)
     #        plt.show()
    
    return pred_, y_, y_obs_, rmse_models
def plot_output(y_pred, y_true, depth_steps, time_label, figsize=(20,10)):
    time_label = np.array([time[:10] for time in time_label])
    time_label = time_label[::depth_steps]
    
    y_pred = y_pred.flatten().detach().cpu().numpy()
    y_true = y_true.flatten().detach().cpu().numpy()
    
    y_true = np.reshape(y_true, (y_true.shape[0]//depth_steps, depth_steps))
    y_pred = np.reshape(y_pred, (y_pred.shape[0]//depth_steps, depth_steps))
    
    N_pts = 6 # number of points to display on the x-label
    
    fig, ax = plt.subplots(3, 1, figsize=figsize)
    sns.heatmap(y_true.T, ax=ax[0], cmap='seismic', vmin=0., vmax=35.)
    ax[0].set_ylabel("Depth", fontsize=15)
    ax[0].set_xlabel("Time", fontsize=15)
    
    xticks_ix = np.array(ax[0].get_xticks()).astype(int)
    time_label = time_label[xticks_ix]
    nelement = len(time_label)//N_pts
    time_label = time_label[::nelement]
    ax[0].xaxis.set_major_locator(plt.MaxNLocator(N_pts))
    ax[0].set_xticklabels(time_label, rotation=0)   
#     ax[0].xaxis.set_major_locator(plt.MultipleLocator(100))
    ax[0].collections[0].colorbar.set_label("Actual Temperature")
    
    sns.heatmap(y_pred.T, ax=ax[1], cmap='seismic', vmin=0., vmax=35.)
    ax[1].set_ylabel("Depth", fontsize=15)
    ax[1].set_xlabel("Time", fontsize=15)
    ax[1].xaxis.set_major_locator(plt.MaxNLocator(N_pts))
    ax[1].set_xticklabels(time_label, rotation=0)
    ax[1].collections[0].colorbar.set_label("Predicted Temperature")
    
    sns.heatmap(np.abs(y_pred.T-y_true.T), ax=ax[2], cmap='viridis')
    ax[2].set_ylabel("Depth", fontsize=15)
    ax[2].set_xlabel("Time", fontsize=15)
    ax[2].xaxis.set_major_locator(plt.MaxNLocator(N_pts))
    ax[2].set_xticklabels(time_label, rotation=0)
    
    ax[2].collections[0].colorbar.set_label("Absolute Error")
    plt.show()
Rollout Results
Training Results
train_y_pred, train_y_true, train_y_obs, train_rmse_models = get_rollout_predictions(heating_model, heat_diff_model, mixing_model, convection_model, ice_model, train_loader, plot = True)

train_rmse = rmse(train_y_pred.flatten(), train_y_true.flatten())
train_rmse_obs = rmse(train_y_pred.flatten(), train_y_obs.flatten())
train_l2 = l2_error(train_y_pred.flatten(), train_y_true.flatten())

print(f"Train RMSE Simulated: {train_rmse}")
print(f"Train RMSE Observed Temp: {train_rmse_obs}")
print(f"Train L2 Error: {train_l2}")
print(f"The RMSEs after each modelling stage: {train_rmse_models.mean(axis=0)}")
RMSE of after m0 0.010144097
RMSE of after m1 0.037624568
RMSE of after m2 0.04029596
RMSE of after m3 0.039802603
RMSE of after m4 0.039970685
RMSE of after m0 0.008860382
RMSE of after m1 0.02663425
RMSE of after m2 0.043961212
RMSE of after m3 0.0441074
RMSE of after m4 0.044129364
RMSE of after m0 0.0072885305
RMSE of after m1 0.02634075
RMSE of after m2 0.02652809
RMSE of after m3 0.026908085
RMSE of after m4 0.026930578
RMSE of after m0 0.0066020973
RMSE of after m1 0.023662489
RMSE of after m2 0.021937968
RMSE of after m3 0.022348044
RMSE of after m4 0.022291481
RMSE of after m0 0.007666328
RMSE of after m1 0.014895804
RMSE of after m2 0.017540516
RMSE of after m3 0.016838346
RMSE of after m4 0.016699383
RMSE of after m0 0.0072736423
RMSE of after m1 0.020800596
RMSE of after m2 0.018833308
RMSE of after m3 0.01928143
RMSE of after m4 0.019185813
RMSE of after m0 0.0071256035
RMSE of after m1 0.033374064
RMSE of after m2 0.03151535
RMSE of after m3 0.03190838
RMSE of after m4 0.03199382
RMSE of after m0 0.006956203
RMSE of after m1 0.028209955
RMSE of after m2 0.059644725
RMSE of after m3 0.060045756
RMSE of after m4 0.060022436
RMSE of after m0 0.006747942
RMSE of after m1 0.031223387
RMSE of after m2 0.040684633
RMSE of after m3 0.040554628
RMSE of after m4 0.040646117
RMSE of after m0 0.011199796
RMSE of after m1 0.035304453
RMSE of after m2 0.038427655
RMSE of after m3 0.038160656
RMSE of after m4 0.03834102
RMSE of after m0 0.0057411348
RMSE of after m1 0.022178326
RMSE of after m2 0.023271767
RMSE of after m3 0.023290046
RMSE of after m4 0.023317778
RMSE of after m0 0.005094319
RMSE of after m1 0.032945037
RMSE of after m2 0.03369337
RMSE of after m3 0.03292564
RMSE of after m4 0.032924272
RMSE of after m0 0.005515631
RMSE of after m1 0.02833733
RMSE of after m2 0.03104878
RMSE of after m3 0.030886766
RMSE of after m4 0.031049982
RMSE of after m0 0.0056879274
RMSE of after m1 0.030704023
RMSE of after m2 0.027875364
RMSE of after m3 0.026513916
RMSE of after m4 0.02668547
RMSE of after m0 0.008925421
RMSE of after m1 0.03684543
RMSE of after m2 0.12146
RMSE of after m3 0.12119544
RMSE of after m4 0.12118212
RMSE of after m0 0.007950872
RMSE of after m1 0.040321704
RMSE of after m2 0.050963107
RMSE of after m3 0.05029201
RMSE of after m4 0.050139144
RMSE of after m0 0.0063130283
RMSE of after m1 0.031969346
RMSE of after m2 0.026268696
RMSE of after m3 0.025310878
RMSE of after m4 0.025138734
RMSE of after m0 0.0043146475
RMSE of after m1 0.02236241
RMSE of after m2 0.02099611
RMSE of after m3 0.020585423
RMSE of after m4 0.020707127
RMSE of after m0 0.004230079
RMSE of after m1 0.025509233
RMSE of after m2 0.026800279
RMSE of after m3 0.026767679
RMSE of after m4 0.02682985
RMSE of after m0 0.004952314
RMSE of after m1 0.03399972
RMSE of after m2 0.040172815
RMSE of after m3 0.038313005
RMSE of after m4 0.03829374
RMSE of after m0 0.00427886
RMSE of after m1 0.02545922
RMSE of after m2 0.03258003
RMSE of after m3 0.032952897
RMSE of after m4 0.03292177
RMSE of after m0 0.0034561465
RMSE of after m1 0.027626665
RMSE of after m2 0.039424594
RMSE of after m3 0.038564652
RMSE of after m4 0.03866498
RMSE of after m0 0.005173788
RMSE of after m1 0.027826853
RMSE of after m2 0.039447777
RMSE of after m3 0.03891129
RMSE of after m4 0.038911358
RMSE of after m0 0.004257567
RMSE of after m1 0.018623866
RMSE of after m2 0.017425504
RMSE of after m3 0.014232408
RMSE of after m4 0.014143155
RMSE of after m0 0.004201052
RMSE of after m1 0.020634295
RMSE of after m2 0.043396775
RMSE of after m3 0.043022312
RMSE of after m4 0.04310336
RMSE of after m0 0.0056763194
RMSE of after m1 0.030298537
RMSE of after m2 0.03089452
RMSE of after m3 0.02844328
RMSE of after m4 0.028496481
RMSE of after m0 0.0060303016
RMSE of after m1 0.023362946
RMSE of after m2 0.019355696
RMSE of after m3 0.01743934
RMSE of after m4 0.017482528
RMSE of after m0 0.0054717497
RMSE of after m1 0.016366575
RMSE of after m2 0.01705233
RMSE of after m3 0.013999959
RMSE of after m4 0.01449731
RMSE of after m0 0.0056925393
RMSE of after m1 0.01479807
RMSE of after m2 0.019528123
RMSE of after m3 0.020008361
RMSE of after m4 0.0202301
RMSE of after m0 0.004974869
RMSE of after m1 0.011674307
RMSE of after m2 0.01802907
RMSE of after m3 0.019113934
RMSE of after m4 0.019088738
RMSE of after m0 0.007479295
RMSE of after m1 0.020037832
RMSE of after m2 0.046467006
RMSE of after m3 0.04698442
RMSE of after m4 0.046977535
RMSE of after m0 0.0063380823
RMSE of after m1 0.015980933
RMSE of after m2 0.024161985
RMSE of after m3 0.025160711
RMSE of after m4 0.02507921
RMSE of after m0 0.007042058
RMSE of after m1 0.017225131
RMSE of after m2 0.017981572
RMSE of after m3 0.018666625
RMSE of after m4 0.018691719
RMSE of after m0 0.007082764
RMSE of after m1 0.020638557
RMSE of after m2 0.017371548
RMSE of after m3 0.015307529
RMSE of after m4 0.015073331
RMSE of after m0 0.008176868
RMSE of after m1 0.019800253
RMSE of after m2 0.021570737
RMSE of after m3 0.021120058
RMSE of after m4 0.020897774
RMSE of after m0 0.076409906
RMSE of after m1 0.07917833
RMSE of after m2 0.12627998
RMSE of after m3 0.12558396
RMSE of after m4 0.12558496
RMSE of after m0 0.031732336
RMSE of after m1 0.042425223
RMSE of after m2 0.06573606
RMSE of after m3 0.06605238
RMSE of after m4 0.06608416
RMSE of after m0 0.03484711
RMSE of after m1 0.04626552
RMSE of after m2 0.053042457
RMSE of after m3 0.052971117
RMSE of after m4 0.052975293
RMSE of after m0 0.02308793
RMSE of after m1 0.038736
RMSE of after m2 0.15585746
RMSE of after m3 0.15597816
RMSE of after m4 0.15593146
RMSE of after m0 0.019237427
RMSE of after m1 0.04025991
RMSE of after m2 0.057006564
RMSE of after m3 0.05721687
RMSE of after m4 0.057269983
RMSE of after m0 0.01983563
RMSE of after m1 0.031415083
RMSE of after m2 0.03139524
RMSE of after m3 0.03075392
RMSE of after m4 0.030801397
RMSE of after m0 0.02029867
RMSE of after m1 0.02816395
RMSE of after m2 0.028739542
RMSE of after m3 0.028956154
RMSE of after m4 0.028987609
RMSE of after m0 0.009014793
RMSE of after m1 0.026538402
RMSE of after m2 0.032498807
RMSE of after m3 0.03347889
RMSE of after m4 0.033525296
RMSE of after m0 0.01063853
RMSE of after m1 0.030852556
RMSE of after m2 0.052343663
RMSE of after m3 0.05259992
RMSE of after m4 0.052616093
RMSE of after m0 0.010554555
RMSE of after m1 0.033141788
RMSE of after m2 0.14295128
RMSE of after m3 0.14284356
RMSE of after m4 0.14285043
RMSE of after m0 0.007116736
RMSE of after m1 0.033559836
RMSE of after m2 0.03895977
RMSE of after m3 0.039127916
RMSE of after m4 0.039111074
RMSE of after m0 0.0117725
RMSE of after m1 0.028880864
RMSE of after m2 0.02991791
RMSE of after m3 0.02990396
RMSE of after m4 0.0297946
RMSE of after m0 0.0084553575
RMSE of after m1 0.036005776
RMSE of after m2 0.09775385
RMSE of after m3 0.097976685
RMSE of after m4 0.09796968
RMSE of after m0 0.006828251
RMSE of after m1 0.032826573
RMSE of after m2 0.041270077
RMSE of after m3 0.041068297
RMSE of after m4 0.041024946
RMSE of after m0 0.008504787
RMSE of after m1 0.028820394
RMSE of after m2 0.052570995
RMSE of after m3 0.052701093
RMSE of after m4 0.05271099
RMSE of after m0 0.006938534
RMSE of after m1 0.03014858
RMSE of after m2 0.090555206
RMSE of after m3 0.09027133
RMSE of after m4 0.09019158
RMSE of after m0 0.0051360647
RMSE of after m1 0.030179985
RMSE of after m2 0.03216788
RMSE of after m3 0.0272392
RMSE of after m4 0.027209576
RMSE of after m0 0.0046432014
RMSE of after m1 0.035918906
RMSE of after m2 0.043359675
RMSE of after m3 0.042157974
RMSE of after m4 0.042067822
RMSE of after m0 0.008325198
RMSE of after m1 0.032525007
RMSE of after m2 0.059147775
RMSE of after m3 0.058906358
RMSE of after m4 0.05895824
RMSE of after m0 0.0067159343
RMSE of after m1 0.02642491
RMSE of after m2 0.030351909
RMSE of after m3 0.03014848
RMSE of after m4 0.030172719
RMSE of after m0 0.0060992106
RMSE of after m1 0.02040458
RMSE of after m2 0.02138083
RMSE of after m3 0.021108847
RMSE of after m4 0.021078086
RMSE of after m0 0.0050072875
RMSE of after m1 0.024847927
RMSE of after m2 0.025971588
RMSE of after m3 0.025764128
RMSE of after m4 0.025739362
RMSE of after m0 0.0050395187
RMSE of after m1 0.029372213
RMSE of after m2 0.029643208
RMSE of after m3 0.029866518
RMSE of after m4 0.029796552
RMSE of after m0 0.004306414
RMSE of after m1 0.019676484
RMSE of after m2 0.01893301
RMSE of after m3 0.019281752
RMSE of after m4 0.019176766
RMSE of after m0 0.004385157
RMSE of after m1 0.026315562
RMSE of after m2 0.053139515
RMSE of after m3 0.053171627
RMSE of after m4 0.053160932
RMSE of after m0 0.006744575
RMSE of after m1 0.03020695
RMSE of after m2 0.07984054
RMSE of after m3 0.07959027
RMSE of after m4 0.079560384
RMSE of after m0 0.0045704436
RMSE of after m1 0.023730993
RMSE of after m2 0.043456517
RMSE of after m3 0.04363915
RMSE of after m4 0.043566685
RMSE of after m0 0.0046229037
RMSE of after m1 0.027460666
RMSE of after m2 0.029309802
RMSE of after m3 0.02878194
RMSE of after m4 0.028728295
RMSE of after m0 0.0050797053
RMSE of after m1 0.023557782
RMSE of after m2 0.029131832
RMSE of after m3 0.029084262
RMSE of after m4 0.029098906
RMSE of after m0 0.0059046852
RMSE of after m1 0.020944161
RMSE of after m2 0.051582407
RMSE of after m3 0.051426634
RMSE of after m4 0.05141422
RMSE of after m0 0.0051268386
RMSE of after m1 0.028332792
RMSE of after m2 0.03820205
RMSE of after m3 0.038023476
RMSE of after m4 0.03806359
RMSE of after m0 0.0054017575
RMSE of after m1 0.02739199
RMSE of after m2 0.023327472
RMSE of after m3 0.022636151
RMSE of after m4 0.02263639
RMSE of after m0 0.00784597
RMSE of after m1 0.026660498
RMSE of after m2 0.05715639
RMSE of after m3 0.05688069
RMSE of after m4 0.056884963
RMSE of after m0 0.008417818
RMSE of after m1 0.030060543
RMSE of after m2 0.026039083
RMSE of after m3 0.025116868
RMSE of after m4 0.025078936
RMSE of after m0 0.008446937
RMSE of after m1 0.02258106
RMSE of after m2 0.020062175
RMSE of after m3 0.019549815
RMSE of after m4 0.019549338
RMSE of after m0 0.039023012
RMSE of after m1 0.042597607
RMSE of after m2 0.04473309
RMSE of after m3 0.044840287
RMSE of after m4 0.044866
RMSE of after m0 0.0153569095
RMSE of after m1 0.038480323
RMSE of after m2 0.046331096
RMSE of after m3 0.04599241
RMSE of after m4 0.046120837
RMSE of after m0 0.01857328
RMSE of after m1 0.04601527
RMSE of after m2 0.069914035
RMSE of after m3 0.06928304
RMSE of after m4 0.0694076
RMSE of after m0 0.026093662
RMSE of after m1 0.042554148
RMSE of after m2 0.049433492
RMSE of after m3 0.049164224
RMSE of after m4 0.049240686
RMSE of after m0 0.011599626
RMSE of after m1 0.045124087
RMSE of after m2 0.04235507
RMSE of after m3 0.040218454
RMSE of after m4 0.040218808
RMSE of after m0 0.0065024197
RMSE of after m1 0.03512642
RMSE of after m2 0.03462986
RMSE of after m3 0.030848037
RMSE of after m4 0.030860167
RMSE of after m0 0.00914799
RMSE of after m1 0.03705198
RMSE of after m2 0.037513733
RMSE of after m3 0.032996234
RMSE of after m4 0.03291362
RMSE of after m0 0.007580307
RMSE of after m1 0.032637816
RMSE of after m2 0.04919482
RMSE of after m3 0.048453398
RMSE of after m4 0.048436612
RMSE of after m0 0.006257987
RMSE of after m1 0.036064893
RMSE of after m2 0.03439277
RMSE of after m3 0.033506285
RMSE of after m4 0.033651546
RMSE of after m0 0.0065950286
RMSE of after m1 0.031100119
RMSE of after m2 0.03617241
RMSE of after m3 0.036293406
RMSE of after m4 0.036340628
RMSE of after m0 0.0071059074
RMSE of after m1 0.025687436
RMSE of after m2 0.030084305
RMSE of after m3 0.030283645
RMSE of after m4 0.030249637
RMSE of after m0 0.010186248
RMSE of after m1 0.028973207
RMSE of after m2 0.09391982
RMSE of after m3 0.09391954
RMSE of after m4 0.093925156
RMSE of after m0 0.0068384204
RMSE of after m1 0.031864315
RMSE of after m2 0.02677139
RMSE of after m3 0.02592662
RMSE of after m4 0.025730787
RMSE of after m0 0.0050783483
RMSE of after m1 0.033664495
RMSE of after m2 0.04696001
RMSE of after m3 0.045747153
RMSE of after m4 0.045717448
RMSE of after m0 0.0040049013
RMSE of after m1 0.031061139
RMSE of after m2 0.03496378
RMSE of after m3 0.03355132
RMSE of after m4 0.0335289
Train RMSE Simulated: 0.05025140941143036
Train RMSE Observed Temp: 3.3953664302825928
Train L2 Error: 0.0032549744937568903
The RMSEs after each modelling stage: [0.00965834 0.02971073 0.04274187 0.04224525 0.0422508 ]
plot_output(train_y_pred, train_y_true, depth_steps, train_time, figsize=(20,10))

plot_output(train_y_pred, train_y_obs, depth_steps, train_time, figsize=(20,10))

Test Results
test_y_pred, test_y_true, test_y_obs, test_rmse_models = get_rollout_predictions(heating_model,heat_diff_model, mixing_model, convection_model, ice_model, test_loader, plot = True)

test_rmse = rmse(test_y_pred.flatten(), test_y_true.flatten())
test_rmse_obs = rmse(test_y_pred.flatten(), test_y_obs.flatten())
test_l2 = l2_error(test_y_pred.flatten(), test_y_true.flatten())

print(f"Test RMSE Simulated: {test_rmse}")
print(f"Test RMSE Observed Temp: {test_rmse_obs}")
print(f"test L2 Error: {test_l2}")
print(f"The RMSEs after each modelling stage: {test_rmse_models.mean(axis=0)}")
RMSE of after m0 0.0052240295
RMSE of after m1 0.032098785
RMSE of after m2 0.05493689
RMSE of after m3 0.054504707
RMSE of after m4 0.054478336
RMSE of after m0 0.013247291
RMSE of after m1 0.041039597
RMSE of after m2 0.06062976
RMSE of after m3 0.0600222
RMSE of after m4 0.06002936
RMSE of after m0 0.018338319
RMSE of after m1 0.03278327
RMSE of after m2 0.050566263
RMSE of after m3 0.050632916
RMSE of after m4 0.050555773
RMSE of after m0 0.009621235
RMSE of after m1 0.039245635
RMSE of after m2 0.04318311
RMSE of after m3 0.042104222
RMSE of after m4 0.04201592
RMSE of after m0 0.009197004
RMSE of after m1 0.03766844
RMSE of after m2 0.03685126
RMSE of after m3 0.035601195
RMSE of after m4 0.03567954
RMSE of after m0 0.011338914
RMSE of after m1 0.030836696
RMSE of after m2 0.0309695
RMSE of after m3 0.029244293
RMSE of after m4 0.029415235
RMSE of after m0 0.011111792
RMSE of after m1 0.032598823
RMSE of after m2 0.029723601
RMSE of after m3 0.028050816
RMSE of after m4 0.028916271
RMSE of after m0 0.017437624
RMSE of after m1 0.03989626
RMSE of after m2 0.06842144
RMSE of after m3 0.068692766
RMSE of after m4 0.0695381
RMSE of after m0 0.015339907
RMSE of after m1 0.035310447
RMSE of after m2 0.035608355
RMSE of after m3 0.03766969
RMSE of after m4 0.039070163
RMSE of after m0 0.021588057
RMSE of after m1 0.04412005
RMSE of after m2 0.039092276
RMSE of after m3 0.036757957
RMSE of after m4 0.037030663
RMSE of after m0 0.015177476
RMSE of after m1 0.03137681
RMSE of after m2 0.04093488
RMSE of after m3 0.04008603
RMSE of after m4 0.04058467
RMSE of after m0 0.01347181
RMSE of after m1 0.03195449
RMSE of after m2 0.04342572
RMSE of after m3 0.04356201
RMSE of after m4 0.043985657
RMSE of after m0 0.027026575
RMSE of after m1 0.04520093
RMSE of after m2 0.062770516
RMSE of after m3 0.06519236
RMSE of after m4 0.066856004
RMSE of after m0 0.19089057
RMSE of after m1 0.19748223
RMSE of after m2 0.24750224
RMSE of after m3 0.25004622
RMSE of after m4 0.2530377
RMSE of after m0 0.0474245
RMSE of after m1 0.05680619
RMSE of after m2 0.07805276
RMSE of after m3 0.0809183
RMSE of after m4 0.08495633
RMSE of after m0 0.023995467
RMSE of after m1 0.040488634
RMSE of after m2 0.049371712
RMSE of after m3 0.057122968
RMSE of after m4 0.06256992
RMSE of after m0 0.037471667
RMSE of after m1 0.045562733
RMSE of after m2 0.04660882
RMSE of after m3 0.057193097
RMSE of after m4 0.06622027
RMSE of after m0 0.039743666
RMSE of after m1 0.056589928
RMSE of after m2 0.06442328
RMSE of after m3 0.07000151
RMSE of after m4 0.071444124
RMSE of after m0 0.01948551
RMSE of after m1 0.03113881
RMSE of after m2 0.03358457
RMSE of after m3 0.043914255
RMSE of after m4 0.048023082
RMSE of after m0 0.028512418
RMSE of after m1 0.036822703
RMSE of after m2 0.04780852
RMSE of after m3 0.054481
RMSE of after m4 0.062251575
RMSE of after m0 0.029333215
RMSE of after m1 0.044185765
RMSE of after m2 0.045627613
RMSE of after m3 0.060727622
RMSE of after m4 0.06515339
RMSE of after m0 0.7708606
RMSE of after m1 0.77433366
RMSE of after m2 0.7609626
RMSE of after m3 0.7550505
RMSE of after m4 0.75549877
RMSE of after m0 0.14570992
RMSE of after m1 0.14852768
RMSE of after m2 0.14440688
RMSE of after m3 0.14555748
RMSE of after m4 0.14457384
RMSE of after m0 0.11066744
RMSE of after m1 0.116690174
RMSE of after m2 0.112775296
RMSE of after m3 0.11285292
RMSE of after m4 0.11161555
RMSE of after m0 0.1874118
RMSE of after m1 0.19054031
RMSE of after m2 0.19031096
RMSE of after m3 0.19093241
RMSE of after m4 0.19005822
RMSE of after m0 0.1444444
RMSE of after m1 0.15027212
RMSE of after m2 0.16925482
RMSE of after m3 0.16987087
RMSE of after m4 0.1695804
RMSE of after m0 0.1561364
RMSE of after m1 0.16032155
RMSE of after m2 0.15925287
RMSE of after m3 0.15956838
RMSE of after m4 0.15874523
RMSE of after m0 0.038919356
RMSE of after m1 0.054296564
RMSE of after m2 0.051244345
RMSE of after m3 0.047785766
RMSE of after m4 0.047171988
RMSE of after m0 0.07861773
RMSE of after m1 0.08756565
RMSE of after m2 0.10883987
RMSE of after m3 0.10915652
RMSE of after m4 0.10906587
RMSE of after m0 0.090155005
RMSE of after m1 0.09455631
RMSE of after m2 0.0945299
RMSE of after m3 0.095305964
RMSE of after m4 0.094540164
RMSE of after m0 0.16702433
RMSE of after m1 0.17679258
RMSE of after m2 0.19623289
RMSE of after m3 0.19585977
RMSE of after m4 0.19543949
RMSE of after m0 0.18694079
RMSE of after m1 0.1938422
RMSE of after m2 0.20631602
RMSE of after m3 0.20625253
RMSE of after m4 0.20611362
RMSE of after m0 0.11743138
RMSE of after m1 0.12012484
RMSE of after m2 0.11956446
RMSE of after m3 0.11940104
RMSE of after m4 0.11883772
RMSE of after m0 0.07375469
RMSE of after m1 0.07850337
RMSE of after m2 0.07968008
RMSE of after m3 0.07968127
RMSE of after m4 0.07902407
RMSE of after m0 0.0495866
RMSE of after m1 0.058433235
RMSE of after m2 0.0664564
RMSE of after m3 0.0673495
RMSE of after m4 0.066875726
RMSE of after m0 0.060722034
RMSE of after m1 0.074135326
RMSE of after m2 0.08493667
RMSE of after m3 0.083230264
RMSE of after m4 0.082982376
RMSE of after m0 0.10930651
RMSE of after m1 0.12013483
RMSE of after m2 0.120282285
RMSE of after m3 0.120517686
RMSE of after m4 0.12033017
RMSE of after m0 0.036553387
RMSE of after m1 0.04538423
RMSE of after m2 0.05799212
RMSE of after m3 0.058936026
RMSE of after m4 0.058650814
RMSE of after m0 0.028518727
RMSE of after m1 0.044170007
RMSE of after m2 0.043155603
RMSE of after m3 0.0439345
RMSE of after m4 0.04338464
RMSE of after m0 0.046752665
RMSE of after m1 0.050968118
RMSE of after m2 0.088666834
RMSE of after m3 0.08936818
RMSE of after m4 0.08902579
RMSE of after m0 0.04142792
RMSE of after m1 0.046389807
RMSE of after m2 0.056673553
RMSE of after m3 0.0573102
RMSE of after m4 0.056872293
RMSE of after m0 0.024427833
RMSE of after m1 0.04026719
RMSE of after m2 0.04556879
RMSE of after m3 0.04569322
RMSE of after m4 0.045973398
RMSE of after m0 0.021718064
RMSE of after m1 0.04009294
RMSE of after m2 0.06938311
RMSE of after m3 0.06795743
RMSE of after m4 0.06828546
RMSE of after m0 0.015843721
RMSE of after m1 0.036237624
RMSE of after m2 0.03171473
RMSE of after m3 0.031181222
RMSE of after m4 0.031400442
RMSE of after m0 0.015004216
RMSE of after m1 0.041767854
RMSE of after m2 0.038594406
RMSE of after m3 0.033061657
RMSE of after m4 0.03321945
RMSE of after m0 0.017037671
RMSE of after m1 0.03469123
RMSE of after m2 0.039760124
RMSE of after m3 0.039410394
RMSE of after m4 0.039491102
RMSE of after m0 0.012051283
RMSE of after m1 0.032584105
RMSE of after m2 0.03098009
RMSE of after m3 0.030607905
RMSE of after m4 0.030873273
RMSE of after m0 0.034174033
RMSE of after m1 0.04210676
RMSE of after m2 0.10447845
RMSE of after m3 0.10446876
RMSE of after m4 0.10434565
RMSE of after m0 0.020168506
RMSE of after m1 0.03860246
RMSE of after m2 0.055730753
RMSE of after m3 0.0560809
RMSE of after m4 0.056242745
RMSE of after m0 0.012888975
RMSE of after m1 0.029280614
RMSE of after m2 0.05712504
RMSE of after m3 0.056699563
RMSE of after m4 0.056939047
RMSE of after m0 0.026050068
RMSE of after m1 0.040220447
RMSE of after m2 0.06784636
RMSE of after m3 0.06706791
RMSE of after m4 0.06707535
RMSE of after m0 0.031146854
RMSE of after m1 0.04310511
RMSE of after m2 0.0446741
RMSE of after m3 0.04485505
RMSE of after m4 0.044271234
RMSE of after m0 0.09550827
RMSE of after m1 0.09886909
RMSE of after m2 0.11830017
RMSE of after m3 0.11816024
RMSE of after m4 0.1183285
RMSE of after m0 0.016336154
RMSE of after m1 0.029435555
RMSE of after m2 0.033079464
RMSE of after m3 0.033062186
RMSE of after m4 0.033101328
RMSE of after m0 0.01850596
RMSE of after m1 0.036397953
RMSE of after m2 0.037749555
RMSE of after m3 0.03483886
RMSE of after m4 0.035244353
RMSE of after m0 0.019443318
RMSE of after m1 0.03221818
RMSE of after m2 0.031746924
RMSE of after m3 0.031568173
RMSE of after m4 0.03181777
RMSE of after m0 0.022882352
RMSE of after m1 0.03480266
RMSE of after m2 0.040352892
RMSE of after m3 0.040345665
RMSE of after m4 0.04065957
Test RMSE Simulated: 0.13542313873767853
Test RMSE Observed Temp: 3.100245475769043
test L2 Error: 0.007726485375314951
The RMSEs after each modelling stage: [0.0640194  0.07754157 0.08717039 0.08788609 0.08855206]
plot_output(test_y_pred, test_y_true, depth_steps, test_time, figsize=(20,10))

plot_output(test_y_pred, test_y_obs, depth_steps, train_time, figsize=(20,10))

Evaluate ALL Models individually
Heat Diffusion Model
train_rmse, train_l2_err = compute_metrics( heat_diff_model, train_loader, m1_input_column_ix, m1_output_column_ix, train_mean, train_std)
print(f"Train RMSE: {train_rmse}, Train L2 Error: {train_l2_err}")

test_rmse, test_l2_err = compute_metrics(heat_diff_model, test_loader, m1_input_column_ix, m1_output_column_ix, train_mean, train_std)
print(f"Test RMSE: {test_rmse}, Train L2 Error: {test_l2_err}")
Train RMSE: 12.827437599999891, Train L2 Error: 0.8309701714641251
Test RMSE: 14.88900707541003, Train L2 Error: 0.8487842305478619
Mixing Model
train_rmse, train_l2_err = compute_metrics(mixing_model, train_loader, m2_input_column_ix, m2_output_column_ix, train_mean, train_std)
print(f"Train RMSE: {train_rmse}, Train L2 Error: {train_l2_err}")

test_rmse, test_l2_err = compute_metrics(mixing_model, test_loader, m2_input_column_ix, m2_output_column_ix, train_mean, train_std)
print(f"Test RMSE: {test_rmse}, Train L2 Error: {test_l2_err}")
Train RMSE: 0.0435148034361539, Train L2 Error: 0.0028188573841423452
Test RMSE: 0.04440403271307789, Train L2 Error: 0.0025312998691790854
Convection Model
train_rmse, train_l2_err = compute_metrics(convection_model, train_loader, m3_input_column_ix, m3_output_column_ix, train_mean, train_std)
print(f"Train RMSE: {train_rmse}, Train L2 Error: {train_l2_err}")

test_rmse, test_l2_err = compute_metrics(convection_model, test_loader, m3_input_column_ix, m3_output_column_ix, train_mean, train_std)
print(f"Test RMSE: {test_rmse}, Train L2 Error: {test_l2_err}")
Train RMSE: 0.006609912617224429, Train L2 Error: 0.00042818981247232937
Test RMSE: 0.011185664355474322, Train L2 Error: 0.0006376559065977617
Ice Model
train_rmse, train_l2_err = compute_metrics(ice_model, train_loader, m4_input_column_ix, m4_output_column_ix, train_mean, train_std)
print(f"Train RMSE: {train_rmse}, Train L2 Error: {train_l2_err}")

test_rmse, test_l2_err = compute_metrics(ice_model, test_loader, m4_input_column_ix, m4_output_column_ix, train_mean, train_std)
print(f"Test RMSE: {test_rmse}, Train L2 Error: {test_l2_err}")
Train RMSE: 0.0008162126385603805, Train L2 Error: 5.287421436283234e-05
Test RMSE: 0.005814208886940472, Train L2 Error: 0.00033144787123317687
 
Finetuning
lr = 1e-5
# decay_rate = 0.1
# decay_steps = 500

params = list(heating_model.parameters()) + list(heat_diff_model.parameters()) + list(mixing_model.parameters()) + list(convection_model.parameters()) + list(ice_model.parameters())

optimizer = torch.optim.Adam(params, lr=lr, 
                             betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=decay_steps, gamma=decay_rate)

criterion = torch.nn.MSELoss()
Freeze model weights
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False

def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True
freeze_model(heating_model)
freeze_model(mixing_model)
freeze_model(convection_model)
freeze_model(ice_model)
unfreeze_model(heat_diff_model)
#unfreeze_model(heating_model)
Finetuning on Observed Temperature
heating_model.train()
heat_diff_model.train()
mixing_model.train()
convection_model.train()
ice_model.train()

n_epochs = 500

# mean and standard dev of "temp_total04" of model 4 output
mean_out4 = torch.tensor(train_mean[m4_output_column_ix[0]]).float().to(device)
std_out4 = torch.tensor(train_std[m4_output_column_ix[0]]).float().to(device)

mean_obs = torch.tensor(train_mean[obs_temp_columns_ix[0]]).float().to(device)
std_obs = torch.tensor(train_std[obs_temp_columns_ix[0]]).float().to(device)

# mean and standard dev of "input_temp" of model 1 input
m1_mean = torch.tensor(train_mean[m1_input_column_ix[-1]]).float().to(device)
m1_std = torch.tensor(train_std[m1_input_column_ix[-1]]).float().to(device)

train_loss = []
LOSS_m0 = []
LOSS_m1 = []
LOSS_m2 = []
LOSS_m3 = []
LOSS_m4 = []

for it in tqdm(range(n_epochs)):
    loss_epoch = 0
    loss_epoch_m0 = 0
    loss_epoch_m1 = 0
    loss_epoch_m2 = 0
    loss_epoch_m3 = 0
    loss_epoch_m4 = 0
    for ix, x in enumerate(iter(train_loader)):
        x = x.to(device).float()
        optimizer.zero_grad()
        
      #  m1_input = x[:, m1_input_column_ix]
        m0_input = x[:, m0_input_column_ix]
        
        #model 0
        m0_pred = heating_model(m0_input) #predicts diff and temp
        loss_m0 = criterion(m0_pred, x[:, m0_output_column_ix])

        #model 1
        m1_input = torch.cat([x[:, m1_input_column_ix[:-1]], m0_pred], dim=-1)
        # m1_pred = heat_diff_model(m1_input)
        
        proj = heat_diff_model(m1_input)
        pred = implicit_diffusion(proj, m0_pred)        
        m1_pred = pred.to(dtype=torch.float32)
                
        loss_m1 = criterion(m1_pred, x[:, m1_output_column_ix])
        
        #m1_pred = heat_diff_model(m1_input) #predicts diff and temp
        #m1_pred_temp = m1_pred[:,1:2]
            
        #loss_m1 = criterion(m1_pred_temp, x[:, m1_output_column_ix[1]].unsqueeze(1))

        #model 2
        m2_input = torch.cat([x[:, m2_input_column_ix[:-1]], m1_pred], dim=-1)
        m2_pred = mixing_model(m2_input)
            
        loss_m2 = criterion(m2_pred, x[:, m2_output_column_ix])

        #model 3
        m3_input = torch.cat([x[:, m3_input_column_ix[:-1]], m2_pred], dim=-1)
        m3_pred = convection_model(m3_input)

        loss_m3 = criterion(m3_pred, x[:, m3_output_column_ix])

        #model 4
        m4_input = torch.cat([x[:, m4_input_column_ix[:-1]], m3_pred], dim=-1)
        m4_pred = ice_model(m4_input)
        
        obs_temp_true = x[:, obs_temp_columns_ix] * std_obs + mean_obs
        obs_temp_true_norm = (obs_temp_true - mean_out4)/std_out4
        
        loss_m4 = criterion(m4_pred, obs_temp_true_norm)

        #loss = (loss_m0 + loss_m1 + loss_m2 + loss_m3 + loss_m4)
        
        loss = loss_m4
        
        loss.backward()
        optimizer.step()
        
        loss_epoch += loss.item()
        loss_epoch_m0 += loss_m0.item()
        loss_epoch_m1 += loss_m1.item()
        loss_epoch_m2 += loss_m2.item()
        loss_epoch_m3 += loss_m3.item()
        loss_epoch_m4 += loss_m4.item()
    
    loss_epoch = loss_epoch/len(train_loader)
    loss_epoch_m0 = loss_epoch_m0/len(train_loader)
    loss_epoch_m1 = loss_epoch_m1/len(train_loader)
    loss_epoch_m2 = loss_epoch_m2/len(train_loader)
    loss_epoch_m3 = loss_epoch_m3/len(train_loader)
    loss_epoch_m4 = loss_epoch_m4/len(train_loader)
    
    train_loss.append(loss_epoch)
    LOSS_m0.append(loss_epoch_m0)
    LOSS_m1.append(loss_epoch_m1)
    LOSS_m2.append(loss_epoch_m2)
    LOSS_m3.append(loss_epoch_m3)
    LOSS_m4.append(loss_epoch_m4)
    if it % 50 == 0:
        print(f"Epoch : {it}, Train_loss: {train_loss[-1]}, Loss m0: {LOSS_m0[-1]}, Loss m1: {LOSS_m1[-1]}, Loss m2: {LOSS_m2[-1]}, Loss m3: {LOSS_m3[-1]}, Loss m4: {LOSS_m4[-1]}")
    
    #plot the loss_m1, m2
  0%|          | 1/500 [00:00<05:56,  1.40it/s]
Epoch : 0, Train_loss: 0.3029380817623699, Loss m0: 4.997979521490111e-06, Loss m1: 2.5403267361124404e-05, Loss m2: 6.720240722001216e-05, Loss m3: 6.633828627266612e-05, Loss m4: 0.3029380817623699
 10%|█         | 51/500 [00:35<05:14,  1.43it/s]
Epoch : 50, Train_loss: 0.3003537968677633, Loss m0: 4.997979521490111e-06, Loss m1: 7.640539213967071e-05, Loss m2: 0.0001204780324769672, Loss m3: 0.00011994337675038397, Loss m4: 0.3003537968677633
 20%|██        | 101/500 [01:18<07:11,  1.08s/it]
Epoch : 100, Train_loss: 0.2989697068929672, Loss m0: 4.997979521490111e-06, Loss m1: 0.00013864725145086755, Loss m2: 0.00018369701078079423, Loss m3: 0.00018353201611556443, Loss m4: 0.2989697068929672
 30%|███       | 151/500 [02:21<06:16,  1.08s/it]
Epoch : 150, Train_loss: 0.29815131758942326, Loss m0: 4.997979521490111e-06, Loss m1: 0.00018384011311675696, Loss m2: 0.00022922884963918477, Loss m3: 0.00022925516292589772, Loss m4: 0.29815131758942326
 40%|████      | 201/500 [03:20<05:40,  1.14s/it]
Epoch : 200, Train_loss: 0.29760971928344054, Loss m0: 4.997979521490111e-06, Loss m1: 0.00021521774897872306, Loss m2: 0.00026075668095419294, Loss m3: 0.0002608637845910647, Loss m4: 0.29760971928344054
 50%|█████     | 251/500 [04:16<05:06,  1.23s/it]
Epoch : 250, Train_loss: 0.29713387331541846, Loss m0: 4.997979521490111e-06, Loss m1: 0.0002403690880301463, Loss m2: 0.0002860918723265914, Loss m3: 0.00028622967280063995, Loss m4: 0.29713387331541846
 60%|██████    | 301/500 [05:13<03:56,  1.19s/it]
Epoch : 300, Train_loss: 0.2966050006010953, Loss m0: 4.997979521490111e-06, Loss m1: 0.0002651129429817528, Loss m2: 0.00031108089349479143, Loss m3: 0.00031119855172137784, Loss m4: 0.2966050006010953
 70%|███████   | 351/500 [06:13<02:32,  1.03s/it]
Epoch : 350, Train_loss: 0.29594507112222557, Loss m0: 4.997979521490111e-06, Loss m1: 0.0002975426948122571, Loss m2: 0.00034390259328681756, Loss m3: 0.000343908228301991, Loss m4: 0.29594507112222557
 80%|████████  | 401/500 [07:00<01:33,  1.06it/s]
Epoch : 400, Train_loss: 0.29511494776781866, Loss m0: 4.997979521490111e-06, Loss m1: 0.0003565885088926948, Loss m2: 0.00040371215539327, Loss m3: 0.00040348761009655017, Loss m4: 0.29511494776781866
 90%|█████████ | 451/500 [07:50<00:47,  1.02it/s]
Epoch : 450, Train_loss: 0.29414109885692596, Loss m0: 4.997979521490111e-06, Loss m1: 0.00047802672317415917, Loss m2: 0.0005267743319136036, Loss m3: 0.0005262222500545356, Loss m4: 0.29414109885692596
100%|██████████| 500/500 [08:38<00:00,  1.04s/it]
plt.figure(figsize=(10,6))
plt.plot(train_loss, label = "Train Loss")
plt.plot(LOSS_m0, label = "Loss M0")
plt.plot(LOSS_m1, label = "Loss M1")
plt.plot(LOSS_m2, label = "Loss M2")
plt.plot(LOSS_m3, label = "Loss M3")
plt.plot(LOSS_m4, label = "Loss M4")
plt.ylabel("Loss", fontsize=15)
plt.xlabel("Epoch", fontsize=15)
plt.yscale("log")
plt.legend(fontsize=15)
plt.show()

Rollout Evaluation After FineTuning
train_y_pred, train_y_true, train_y_obs, train_rmse_models = get_rollout_predictions(heating_model, heat_diff_model, mixing_model, convection_model, ice_model, train_loader, plot = True)

train_rmse = rmse(train_y_pred.flatten(), train_y_true.flatten())
train_rmse_obs = rmse(train_y_pred.flatten(), train_y_obs.flatten())
train_l2 = l2_error(train_y_pred.flatten(), train_y_true.flatten())

print(f"Train RMSE Simulated: {train_rmse}")
print(f"Train RMSE Observed Temp: {train_rmse_obs}")
print(f"Train L2 Error: {train_l2}")
print(f"The RMSEs after each modelling stage: {train_rmse_models.mean(axis=0)}")
RMSE of after m0 0.010144097
RMSE of after m1 0.115007706
RMSE of after m2 0.116115384
RMSE of after m3 0.11537405
RMSE of after m4 0.11480767
RMSE of after m0 0.008860382
RMSE of after m1 0.122942954
RMSE of after m2 0.12981623
RMSE of after m3 0.129619
RMSE of after m4 0.12900528
RMSE of after m0 0.0072885305
RMSE of after m1 0.1096832
RMSE of after m2 0.10999011
RMSE of after m3 0.10997454
RMSE of after m4 0.10922474
RMSE of after m0 0.0066020973
RMSE of after m1 0.10672279
RMSE of after m2 0.10694727
RMSE of after m3 0.106760554
RMSE of after m4 0.10617814
RMSE of after m0 0.007666328
RMSE of after m1 0.10571253
RMSE of after m2 0.10656259
RMSE of after m3 0.10638819
RMSE of after m4 0.10576001
RMSE of after m0 0.0072736423
RMSE of after m1 0.106213875
RMSE of after m2 0.10634249
RMSE of after m3 0.10651622
RMSE of after m4 0.10583589
RMSE of after m0 0.0071256035
RMSE of after m1 0.11171851
RMSE of after m2 0.11170267
RMSE of after m3 0.11178547
RMSE of after m4 0.11138629
RMSE of after m0 0.006956203
RMSE of after m1 0.13008288
RMSE of after m2 0.14452228
RMSE of after m3 0.14476883
RMSE of after m4 0.14443827
RMSE of after m0 0.006747942
RMSE of after m1 0.14369841
RMSE of after m2 0.14696291
RMSE of after m3 0.14690435
RMSE of after m4 0.14659296
RMSE of after m0 0.011199796
RMSE of after m1 0.15652752
RMSE of after m2 0.15507407
RMSE of after m3 0.15475252
RMSE of after m4 0.15460452
RMSE of after m0 0.0057411348
RMSE of after m1 0.14072384
RMSE of after m2 0.14041302
RMSE of after m3 0.1403074
RMSE of after m4 0.14010364
RMSE of after m0 0.005094319
RMSE of after m1 0.15001854
RMSE of after m2 0.14872521
RMSE of after m3 0.14833248
RMSE of after m4 0.14815
RMSE of after m0 0.005515631
RMSE of after m1 0.15535234
RMSE of after m2 0.15417944
RMSE of after m3 0.15433247
RMSE of after m4 0.15422884
RMSE of after m0 0.0056879274
RMSE of after m1 0.16166759
RMSE of after m2 0.1591189
RMSE of after m3 0.15917628
RMSE of after m4 0.15908252
RMSE of after m0 0.008925421
RMSE of after m1 0.45601836
RMSE of after m2 0.48644686
RMSE of after m3 0.48634094
RMSE of after m4 0.48605207
RMSE of after m0 0.007950872
RMSE of after m1 0.25732008
RMSE of after m2 0.25753853
RMSE of after m3 0.2570771
RMSE of after m4 0.2570043
RMSE of after m0 0.0063130283
RMSE of after m1 0.17238939
RMSE of after m2 0.17194349
RMSE of after m3 0.17200994
RMSE of after m4 0.17193991
RMSE of after m0 0.0043146475
RMSE of after m1 0.14989221
RMSE of after m2 0.15025467
RMSE of after m3 0.15042934
RMSE of after m4 0.15043472
RMSE of after m0 0.004230079
RMSE of after m1 0.13034914
RMSE of after m2 0.13157919
RMSE of after m3 0.13176323
RMSE of after m4 0.13173515
RMSE of after m0 0.004952314
RMSE of after m1 0.15482506
RMSE of after m2 0.15837015
RMSE of after m3 0.15824258
RMSE of after m4 0.1582624
RMSE of after m0 0.00427886
RMSE of after m1 0.1417484
RMSE of after m2 0.14347231
RMSE of after m3 0.14386728
RMSE of after m4 0.14390996
RMSE of after m0 0.0034561465
RMSE of after m1 0.13825282
RMSE of after m2 0.14221662
RMSE of after m3 0.14233138
RMSE of after m4 0.14238049
RMSE of after m0 0.005173788
RMSE of after m1 0.14870553
RMSE of after m2 0.15337417
RMSE of after m3 0.15360305
RMSE of after m4 0.15368485
RMSE of after m0 0.004257567
RMSE of after m1 0.12521899
RMSE of after m2 0.12622914
RMSE of after m3 0.12629855
RMSE of after m4 0.1263382
RMSE of after m0 0.004201052
RMSE of after m1 0.1298474
RMSE of after m2 0.13776183
RMSE of after m3 0.13805754
RMSE of after m4 0.1381332
RMSE of after m0 0.0056763194
RMSE of after m1 0.14168125
RMSE of after m2 0.14246932
RMSE of after m3 0.14240271
RMSE of after m4 0.14249703
RMSE of after m0 0.0060303016
RMSE of after m1 0.12501404
RMSE of after m2 0.12505108
RMSE of after m3 0.12532116
RMSE of after m4 0.12535183
RMSE of after m0 0.0054717497
RMSE of after m1 0.11268247
RMSE of after m2 0.11393484
RMSE of after m3 0.11409753
RMSE of after m4 0.11417174
RMSE of after m0 0.0056925393
RMSE of after m1 0.11252834
RMSE of after m2 0.11447473
RMSE of after m3 0.1151095
RMSE of after m4 0.11515602
RMSE of after m0 0.004974869
RMSE of after m1 0.11148984
RMSE of after m2 0.11398207
RMSE of after m3 0.11477361
RMSE of after m4 0.11476348
RMSE of after m0 0.007479295
RMSE of after m1 0.1210641
RMSE of after m2 0.1298875
RMSE of after m3 0.13056082
RMSE of after m4 0.13056858
RMSE of after m0 0.0063380823
RMSE of after m1 0.12611853
RMSE of after m2 0.12814766
RMSE of after m3 0.12881154
RMSE of after m4 0.12882604
RMSE of after m0 0.007042058
RMSE of after m1 0.11430944
RMSE of after m2 0.115436114
RMSE of after m3 0.116180286
RMSE of after m4 0.11623454
RMSE of after m0 0.007082764
RMSE of after m1 0.11356256
RMSE of after m2 0.11416314
RMSE of after m3 0.114519194
RMSE of after m4 0.114586376
RMSE of after m0 0.008176868
RMSE of after m1 0.11352636
RMSE of after m2 0.1148162
RMSE of after m3 0.11525334
RMSE of after m4 0.11533157
RMSE of after m0 0.076409906
RMSE of after m1 0.15979943
RMSE of after m2 0.1863846
RMSE of after m3 0.1859015
RMSE of after m4 0.1856499
RMSE of after m0 0.031732336
RMSE of after m1 0.14086084
RMSE of after m2 0.15114489
RMSE of after m3 0.1508852
RMSE of after m4 0.15039858
RMSE of after m0 0.03484711
RMSE of after m1 0.14620332
RMSE of after m2 0.1477048
RMSE of after m3 0.14733423
RMSE of after m4 0.14677542
RMSE of after m0 0.02308793
RMSE of after m1 0.19794944
RMSE of after m2 0.26199898
RMSE of after m3 0.26195908
RMSE of after m4 0.26160505
RMSE of after m0 0.019237427
RMSE of after m1 0.13679077
RMSE of after m2 0.14357577
RMSE of after m3 0.1434542
RMSE of after m4 0.14305
RMSE of after m0 0.01983563
RMSE of after m1 0.14422661
RMSE of after m2 0.14337188
RMSE of after m3 0.14277034
RMSE of after m4 0.14234705
RMSE of after m0 0.02029867
RMSE of after m1 0.13345191
RMSE of after m2 0.13208108
RMSE of after m3 0.1315613
RMSE of after m4 0.13109939
RMSE of after m0 0.009014793
RMSE of after m1 0.12769942
RMSE of after m2 0.12820752
RMSE of after m3 0.12841387
RMSE of after m4 0.1280917
RMSE of after m0 0.01063853
RMSE of after m1 0.14051393
RMSE of after m2 0.14873222
RMSE of after m3 0.14897552
RMSE of after m4 0.14867966
RMSE of after m0 0.010554555
RMSE of after m1 0.24491717
RMSE of after m2 0.28443626
RMSE of after m3 0.28443515
RMSE of after m4 0.28413627
RMSE of after m0 0.007116736
RMSE of after m1 0.18125087
RMSE of after m2 0.18225926
RMSE of after m3 0.18229565
RMSE of after m4 0.18202785
RMSE of after m0 0.0117725
RMSE of after m1 0.14895765
RMSE of after m2 0.14730692
RMSE of after m3 0.14732212
RMSE of after m4 0.14703967
RMSE of after m0 0.0084553575
RMSE of after m1 0.2002813
RMSE of after m2 0.22167481
RMSE of after m3 0.2216452
RMSE of after m4 0.22141553
RMSE of after m0 0.006828251
RMSE of after m1 0.17653884
RMSE of after m2 0.17906831
RMSE of after m3 0.17900184
RMSE of after m4 0.17876925
RMSE of after m0 0.008504787
RMSE of after m1 0.16677332
RMSE of after m2 0.17230554
RMSE of after m3 0.17231308
RMSE of after m4 0.17214398
RMSE of after m0 0.006938534
RMSE of after m1 0.23190516
RMSE of after m2 0.24922414
RMSE of after m3 0.24899183
RMSE of after m4 0.24879283
RMSE of after m0 0.0051360647
RMSE of after m1 0.15774137
RMSE of after m2 0.15781814
RMSE of after m3 0.1569956
RMSE of after m4 0.15682848
RMSE of after m0 0.0046432014
RMSE of after m1 0.15791598
RMSE of after m2 0.16074987
RMSE of after m3 0.16056176
RMSE of after m4 0.16041912
RMSE of after m0 0.008325198
RMSE of after m1 0.17271669
RMSE of after m2 0.17971234
RMSE of after m3 0.17949589
RMSE of after m4 0.17942363
RMSE of after m0 0.0067159343
RMSE of after m1 0.16307178
RMSE of after m2 0.16393954
RMSE of after m3 0.16402085
RMSE of after m4 0.16393642
RMSE of after m0 0.0060992106
RMSE of after m1 0.15004398
RMSE of after m2 0.15031278
RMSE of after m3 0.15027852
RMSE of after m4 0.150274
RMSE of after m0 0.0050072875
RMSE of after m1 0.1474545
RMSE of after m2 0.14753892
RMSE of after m3 0.14756858
RMSE of after m4 0.14758989
RMSE of after m0 0.0050395187
RMSE of after m1 0.13606386
RMSE of after m2 0.13580324
RMSE of after m3 0.13638636
RMSE of after m4 0.13635069
RMSE of after m0 0.004306414
RMSE of after m1 0.12938878
RMSE of after m2 0.12959695
RMSE of after m3 0.13020112
RMSE of after m4 0.13018458
RMSE of after m0 0.004385157
RMSE of after m1 0.14511383
RMSE of after m2 0.15349443
RMSE of after m3 0.15387805
RMSE of after m4 0.15389337
RMSE of after m0 0.006744575
RMSE of after m1 0.17513005
RMSE of after m2 0.19323333
RMSE of after m3 0.19344449
RMSE of after m4 0.1934418
RMSE of after m0 0.0045704436
RMSE of after m1 0.16610494
RMSE of after m2 0.17001346
RMSE of after m3 0.17037266
RMSE of after m4 0.17036763
RMSE of after m0 0.0046229037
RMSE of after m1 0.16726376
RMSE of after m2 0.16830017
RMSE of after m3 0.1684878
RMSE of after m4 0.16849382
RMSE of after m0 0.0050797053
RMSE of after m1 0.14696155
RMSE of after m2 0.14879213
RMSE of after m3 0.14915049
RMSE of after m4 0.14918517
RMSE of after m0 0.0059046852
RMSE of after m1 0.1512799
RMSE of after m2 0.16080382
RMSE of after m3 0.16112302
RMSE of after m4 0.1611484
RMSE of after m0 0.0051268386
RMSE of after m1 0.15602696
RMSE of after m2 0.15802842
RMSE of after m3 0.1582298
RMSE of after m4 0.158254
RMSE of after m0 0.0054017575
RMSE of after m1 0.14160998
RMSE of after m2 0.14193149
RMSE of after m3 0.14214307
RMSE of after m4 0.14216651
RMSE of after m0 0.00784597
RMSE of after m1 0.14748248
RMSE of after m2 0.15650171
RMSE of after m3 0.1565621
RMSE of after m4 0.1565699
RMSE of after m0 0.008417818
RMSE of after m1 0.15605451
RMSE of after m2 0.1555974
RMSE of after m3 0.15553837
RMSE of after m4 0.15551305
RMSE of after m0 0.008446937
RMSE of after m1 0.14042582
RMSE of after m2 0.1415307
RMSE of after m3 0.14167927
RMSE of after m4 0.14172545
RMSE of after m0 0.039023012
RMSE of after m1 0.14005168
RMSE of after m2 0.14182758
RMSE of after m3 0.1416105
RMSE of after m4 0.14160918
RMSE of after m0 0.0153569095
RMSE of after m1 0.117135085
RMSE of after m2 0.120590925
RMSE of after m3 0.11997351
RMSE of after m4 0.11910103
RMSE of after m0 0.01857328
RMSE of after m1 0.15261932
RMSE of after m2 0.16869675
RMSE of after m3 0.16782919
RMSE of after m4 0.16742209
RMSE of after m0 0.026093662
RMSE of after m1 0.19436741
RMSE of after m2 0.1943646
RMSE of after m3 0.19350746
RMSE of after m4 0.19318989
RMSE of after m0 0.011599626
RMSE of after m1 0.1943935
RMSE of after m2 0.19075033
RMSE of after m3 0.18958487
RMSE of after m4 0.18918471
RMSE of after m0 0.0065024197
RMSE of after m1 0.16841671
RMSE of after m2 0.16635521
RMSE of after m3 0.16487016
RMSE of after m4 0.16445391
RMSE of after m0 0.00914799
RMSE of after m1 0.1725377
RMSE of after m2 0.17256424
RMSE of after m3 0.17119475
RMSE of after m4 0.17080267
RMSE of after m0 0.007580307
RMSE of after m1 0.23816726
RMSE of after m2 0.24036208
RMSE of after m3 0.2397175
RMSE of after m4 0.23945117
RMSE of after m0 0.006257987
RMSE of after m1 0.15632623
RMSE of after m2 0.15516734
RMSE of after m3 0.15453792
RMSE of after m4 0.15420079
RMSE of after m0 0.0065950286
RMSE of after m1 0.1704157
RMSE of after m2 0.16786003
RMSE of after m3 0.16756561
RMSE of after m4 0.16731304
RMSE of after m0 0.0071059074
RMSE of after m1 0.1671818
RMSE of after m2 0.16747597
RMSE of after m3 0.1672333
RMSE of after m4 0.16692232
RMSE of after m0 0.010186248
RMSE of after m1 0.25273818
RMSE of after m2 0.27007735
RMSE of after m3 0.26963058
RMSE of after m4 0.2694821
RMSE of after m0 0.0068384204
RMSE of after m1 0.17199849
RMSE of after m2 0.16879936
RMSE of after m3 0.1684954
RMSE of after m4 0.16821893
RMSE of after m0 0.0050783483
RMSE of after m1 0.20034651
RMSE of after m2 0.2033361
RMSE of after m3 0.20292829
RMSE of after m4 0.20275372
RMSE of after m0 0.0040049013
RMSE of after m1 0.167011
RMSE of after m2 0.16687474
RMSE of after m3 0.16640797
RMSE of after m4 0.16624194
Train RMSE Simulated: 0.1683911234140396
Train RMSE Observed Temp: 3.340033531188965
Train L2 Error: 0.010928550735116005
The RMSEs after each modelling stage: [0.00965834 0.15602699 0.16066236 0.16059421 0.16041055]
plot_output(train_y_pred, train_y_true, depth_steps, train_time, figsize=(20,10))

plot_output(train_y_pred, train_y_obs, depth_steps, train_time, figsize=(20,10))

test_y_pred, test_y_true, test_y_obs, test_rmse_models = get_rollout_predictions(heating_model, heat_diff_model, mixing_model, convection_model, ice_model, test_loader, plot = True)

test_rmse = rmse(test_y_pred.flatten(), test_y_true.flatten())
test_rmse_obs = rmse(test_y_pred.flatten(), test_y_obs.flatten())
test_l2 = l2_error(test_y_pred.flatten(), test_y_true.flatten())

print(f"Test RMSE Simulated: {test_rmse}")
print(f"Test RMSE Observed Temp: {test_rmse_obs}")
print(f"test L2 Error: {test_l2}")
print(f"The RMSEs after each modelling stage: {test_rmse_models.mean(axis=0)}")
RMSE of after m0 0.0052240295
RMSE of after m1 0.2080348
RMSE of after m2 0.21332149
RMSE of after m3 0.21314588
RMSE of after m4 0.21307279
RMSE of after m0 0.013247291
RMSE of after m1 0.17860928
RMSE of after m2 0.18490686
RMSE of after m3 0.18477805
RMSE of after m4 0.184738
RMSE of after m0 0.018338319
RMSE of after m1 0.18449013
RMSE of after m2 0.18572359
RMSE of after m3 0.1857395
RMSE of after m4 0.18566437
RMSE of after m0 0.009621235
RMSE of after m1 0.19474837
RMSE of after m2 0.19712329
RMSE of after m3 0.19705388
RMSE of after m4 0.19704773
RMSE of after m0 0.009197004
RMSE of after m1 0.18546568
RMSE of after m2 0.1872291
RMSE of after m3 0.18707523
RMSE of after m4 0.18725944
RMSE of after m0 0.011338914
RMSE of after m1 0.17155132
RMSE of after m2 0.172616
RMSE of after m3 0.17233512
RMSE of after m4 0.17264614
RMSE of after m0 0.011111792
RMSE of after m1 0.17602322
RMSE of after m2 0.1774939
RMSE of after m3 0.17748229
RMSE of after m4 0.1781254
RMSE of after m0 0.017437624
RMSE of after m1 0.45715752
RMSE of after m2 0.4609387
RMSE of after m3 0.46121636
RMSE of after m4 0.46057823
RMSE of after m0 0.015339907
RMSE of after m1 0.18716267
RMSE of after m2 0.18607944
RMSE of after m3 0.18660064
RMSE of after m4 0.18738872
RMSE of after m0 0.021588057
RMSE of after m1 0.18833119
RMSE of after m2 0.18638378
RMSE of after m3 0.1860818
RMSE of after m4 0.18651162
RMSE of after m0 0.015177476
RMSE of after m1 0.21927498
RMSE of after m2 0.22136588
RMSE of after m3 0.2214048
RMSE of after m4 0.22166084
RMSE of after m0 0.01347181
RMSE of after m1 0.21304332
RMSE of after m2 0.2149261
RMSE of after m3 0.21508676
RMSE of after m4 0.21534851
RMSE of after m0 0.027026575
RMSE of after m1 0.21898703
RMSE of after m2 0.22182319
RMSE of after m3 0.22272813
RMSE of after m4 0.22373258
RMSE of after m0 0.19089057
RMSE of after m1 1.201656
RMSE of after m2 1.2063829
RMSE of after m3 1.2084755
RMSE of after m4 1.2027119
RMSE of after m0 0.0474245
RMSE of after m1 0.3257656
RMSE of after m2 0.32987648
RMSE of after m3 0.3303238
RMSE of after m4 0.33166087
RMSE of after m0 0.023995467
RMSE of after m1 0.22002694
RMSE of after m2 0.22235866
RMSE of after m3 0.2244132
RMSE of after m4 0.22673166
RMSE of after m0 0.037471667
RMSE of after m1 0.22606388
RMSE of after m2 0.22846517
RMSE of after m3 0.23117478
RMSE of after m4 0.23510426
RMSE of after m0 0.039743666
RMSE of after m1 0.2088909
RMSE of after m2 0.21244054
RMSE of after m3 0.21381791
RMSE of after m4 0.21563631
RMSE of after m0 0.01948551
RMSE of after m1 0.21557695
RMSE of after m2 0.21640906
RMSE of after m3 0.21808875
RMSE of after m4 0.22015254
RMSE of after m0 0.028512418
RMSE of after m1 0.22154662
RMSE of after m2 0.22568496
RMSE of after m3 0.22713044
RMSE of after m4 0.2303756
RMSE of after m0 0.029333215
RMSE of after m1 0.21501452
RMSE of after m2 0.2108149
RMSE of after m3 0.21391757
RMSE of after m4 0.2162208
RMSE of after m0 0.7708606
RMSE of after m1 0.7886312
RMSE of after m2 0.77584517
RMSE of after m3 0.7698807
RMSE of after m4 0.7704372
RMSE of after m0 0.14570992
RMSE of after m1 0.18208237
RMSE of after m2 0.17915946
RMSE of after m3 0.17939958
RMSE of after m4 0.17792925
RMSE of after m0 0.11066744
RMSE of after m1 0.15743634
RMSE of after m2 0.154671
RMSE of after m3 0.15379688
RMSE of after m4 0.15244304
RMSE of after m0 0.1874118
RMSE of after m1 0.22656436
RMSE of after m2 0.22578909
RMSE of after m3 0.22592254
RMSE of after m4 0.22477582
RMSE of after m0 0.1444444
RMSE of after m1 0.24751876
RMSE of after m2 0.26536292
RMSE of after m3 0.2653401
RMSE of after m4 0.26486948
RMSE of after m0 0.1561364
RMSE of after m1 0.2462111
RMSE of after m2 0.24143226
RMSE of after m3 0.24125105
RMSE of after m4 0.24041988
RMSE of after m0 0.038919356
RMSE of after m1 0.17902051
RMSE of after m2 0.17663433
RMSE of after m3 0.17534801
RMSE of after m4 0.17479901
RMSE of after m0 0.07861773
RMSE of after m1 0.22019678
RMSE of after m2 0.23853196
RMSE of after m3 0.23858464
RMSE of after m4 0.23828739
RMSE of after m0 0.090155005
RMSE of after m1 0.17804319
RMSE of after m2 0.1760924
RMSE of after m3 0.17623542
RMSE of after m4 0.17556818
RMSE of after m0 0.16702433
RMSE of after m1 0.2754767
RMSE of after m2 0.29012436
RMSE of after m3 0.2897754
RMSE of after m4 0.28931442
RMSE of after m0 0.18694079
RMSE of after m1 0.32779455
RMSE of after m2 0.33280182
RMSE of after m3 0.3329177
RMSE of after m4 0.33269966
RMSE of after m0 0.11743138
RMSE of after m1 0.23486073
RMSE of after m2 0.23366868
RMSE of after m3 0.2337098
RMSE of after m4 0.2332324
RMSE of after m0 0.07375469
RMSE of after m1 0.16939665
RMSE of after m2 0.17130199
RMSE of after m3 0.17164348
RMSE of after m4 0.17115173
RMSE of after m0 0.0495866
RMSE of after m1 0.20408136
RMSE of after m2 0.20654598
RMSE of after m3 0.20737039
RMSE of after m4 0.2070785
RMSE of after m0 0.060722034
RMSE of after m1 0.19747157
RMSE of after m2 0.20970213
RMSE of after m3 0.20925723
RMSE of after m4 0.20908287
RMSE of after m0 0.10930651
RMSE of after m1 0.47364464
RMSE of after m2 0.4746794
RMSE of after m3 0.47494027
RMSE of after m4 0.47476166
RMSE of after m0 0.036553387
RMSE of after m1 0.14976558
RMSE of after m2 0.15828128
RMSE of after m3 0.15933664
RMSE of after m4 0.15925643
RMSE of after m0 0.028518727
RMSE of after m1 0.1398909
RMSE of after m2 0.14061835
RMSE of after m3 0.14116932
RMSE of after m4 0.1410899
RMSE of after m0 0.046752665
RMSE of after m1 0.28785273
RMSE of after m2 0.30072725
RMSE of after m3 0.3013959
RMSE of after m4 0.3013666
RMSE of after m0 0.04142792
RMSE of after m1 0.40334505
RMSE of after m2 0.40699008
RMSE of after m3 0.40767822
RMSE of after m4 0.4075091
RMSE of after m0 0.024427833
RMSE of after m1 0.45463333
RMSE of after m2 0.4546195
RMSE of after m3 0.45523694
RMSE of after m4 0.45414266
RMSE of after m0 0.021718064
RMSE of after m1 0.18856435
RMSE of after m2 0.20256357
RMSE of after m3 0.2027073
RMSE of after m4 0.20315503
RMSE of after m0 0.015843721
RMSE of after m1 0.1608827
RMSE of after m2 0.15941112
RMSE of after m3 0.16001892
RMSE of after m4 0.16036132
RMSE of after m0 0.015004216
RMSE of after m1 0.1491604
RMSE of after m2 0.149689
RMSE of after m3 0.14906028
RMSE of after m4 0.1493433
RMSE of after m0 0.017037671
RMSE of after m1 0.16416207
RMSE of after m2 0.16671577
RMSE of after m3 0.1672261
RMSE of after m4 0.16749184
RMSE of after m0 0.012051283
RMSE of after m1 0.15593682
RMSE of after m2 0.15622792
RMSE of after m3 0.15681723
RMSE of after m4 0.15721846
RMSE of after m0 0.034174033
RMSE of after m1 0.87022847
RMSE of after m2 0.87304366
RMSE of after m3 0.8734434
RMSE of after m4 0.8654587
RMSE of after m0 0.020168506
RMSE of after m1 0.3612958
RMSE of after m2 0.3639971
RMSE of after m3 0.3645627
RMSE of after m4 0.36462548
RMSE of after m0 0.012888975
RMSE of after m1 0.1863524
RMSE of after m2 0.19572254
RMSE of after m3 0.19588621
RMSE of after m4 0.19631003
RMSE of after m0 0.026050068
RMSE of after m1 0.20816623
RMSE of after m2 0.216388
RMSE of after m3 0.21647733
RMSE of after m4 0.21659657
RMSE of after m0 0.031146854
RMSE of after m1 0.20804088
RMSE of after m2 0.20765565
RMSE of after m3 0.20804392
RMSE of after m4 0.2081619
RMSE of after m0 0.09550827
RMSE of after m1 0.34228465
RMSE of after m2 0.35183427
RMSE of after m3 0.35182703
RMSE of after m4 0.35201573
RMSE of after m0 0.016336154
RMSE of after m1 0.19509342
RMSE of after m2 0.19506706
RMSE of after m3 0.19553791
RMSE of after m4 0.19571516
RMSE of after m0 0.01850596
RMSE of after m1 0.17746036
RMSE of after m2 0.17982696
RMSE of after m3 0.17982708
RMSE of after m4 0.18004313
RMSE of after m0 0.019443318
RMSE of after m1 0.17720445
RMSE of after m2 0.17831044
RMSE of after m3 0.17893429
RMSE of after m4 0.17918947
RMSE of after m0 0.022882352
RMSE of after m1 0.17765972
RMSE of after m2 0.18033594
RMSE of after m3 0.18089192
RMSE of after m4 0.18110855
Test RMSE Simulated: 0.324886679649353
Test RMSE Observed Temp: 3.027254819869995
test L2 Error: 0.018595343455672264
The RMSEs after each modelling stage: [0.0640194  0.26462863 0.2675918  0.26788635 0.26788383]
plot_output(test_y_pred, test_y_true, depth_steps, test_time, figsize=(20,10))

plot_output(test_y_pred, test_y_obs, depth_steps, train_time, figsize=(20,10))

Evaluate ALL Models individually
Heat Diffusion Model
train_rmse, train_l2_err = compute_metrics(heat_diff_model, train_loader, m1_input_column_ix, m1_output_column_ix, train_mean, train_std)
print(f"Train RMSE: {train_rmse}, Train L2 Error: {train_l2_err}")

test_rmse, test_l2_err = compute_metrics(heat_diff_model, test_loader, m1_input_column_ix, m1_output_column_ix, train_mean, train_std)
print(f"Test RMSE: {test_rmse}, Train L2 Error: {test_l2_err}")
Train RMSE: 33.126759296299674, Train L2 Error: 2.1459740994956915
Test RMSE: 45.79279275387178, Train L2 Error: 2.6105300484694927
Mixing Model
train_rmse, train_l2_err = compute_metrics(mixing_model, train_loader, m2_input_column_ix, m2_output_column_ix, train_mean, train_std)
print(f"Train RMSE: {train_rmse}, Train L2 Error: {train_l2_err}")

test_rmse, test_l2_err = compute_metrics(mixing_model, test_loader, m2_input_column_ix, m2_output_column_ix, train_mean, train_std)
print(f"Test RMSE: {test_rmse}, Train L2 Error: {test_l2_err}")
Train RMSE: 0.0435148034361539, Train L2 Error: 0.0028188573841423452
Test RMSE: 0.04440403271307789, Train L2 Error: 0.0025312998691790854
Convection Model
train_rmse, train_l2_err = compute_metrics(convection_model, train_loader, m3_input_column_ix, m3_output_column_ix, train_mean, train_std)
print(f"Train RMSE: {train_rmse}, Train L2 Error: {train_l2_err}")

test_rmse, test_l2_err = compute_metrics(convection_model, test_loader, m3_input_column_ix, m3_output_column_ix, train_mean, train_std)
print(f"Test RMSE: {test_rmse}, Train L2 Error: {test_l2_err}")
Train RMSE: 0.006609912617224429, Train L2 Error: 0.00042818981247232937
Test RMSE: 0.011185664355474322, Train L2 Error: 0.0006376559065977617
Ice Model
train_rmse, train_l2_err = compute_metrics(ice_model, train_loader, m4_input_column_ix, m4_output_column_ix, train_mean, train_std)
print(f"Train RMSE: {train_rmse}, Train L2 Error: {train_l2_err}")

test_rmse, test_l2_err = compute_metrics(ice_model, test_loader, m4_input_column_ix, m4_output_column_ix, train_mean, train_std)
print(f"Test RMSE: {test_rmse}, Train L2 Error: {test_l2_err}")
Train RMSE: 0.0008162126385603805, Train L2 Error: 5.287421436283234e-05
Test RMSE: 0.005814208886940472, Train L2 Error: 0.00033144787123317687
Compute Model Similarity after Finetuning
def compute_model_similarity(model1, model2):
    weight1 = []
    weight2 = []
    for param1, param2 in zip(model1.parameters(), model2.parameters()):
        weight1.append(param1.detach().clone().flatten())
        weight2.append(param2.detach().clone().flatten())
    weight1 = torch.cat(weight1, dim=0)
    weight2 = torch.cat(weight2, dim=0)
    
    #Cosine Similarity
    cos = nn.CosineSimilarity(dim=0, eps=1e-8)
    cos_sim = cos(weight1, weight2)
    
    #L2 norm
    l2 = torch.norm((weight1-weight2), p='fro', dim=0)
    
    return cos_sim, l2
heat_diff_model_init = MLP(m1_layers, activation="gelu")
heat_diff_model_init.load_state_dict(m1_checkpoint)
heat_diff_model_init = heat_diff_model_init.to(device)

cos_sim, l2_dist = compute_model_similarity(heat_diff_model_init, heat_diff_model)

print(f"Cosine Similarity: {cos_sim}")
print(f"L2 Norm: {l2_dist}")
Initializing Network with Xavier Initialization..
Cosine Similarity: 0.8990045785903931
L2 Norm: 5.173426151275635
mixing_model_init = MLP(m2_layers, activation="gelu")
mixing_model_init.load_state_dict(m2_checkpoint)
mixing_model_init = mixing_model_init.to(device)

cos_sim, l2_dist = compute_model_similarity(mixing_model_init, mixing_model)

print(f"Cosine Similarity: {cos_sim}")
print(f"L2 Norm: {l2_dist}")
Initializing Network with Xavier Initialization..
Cosine Similarity: 1.0
L2 Norm: 0.0
convection_model_init = MLP(m3_layers, activation="gelu")
convection_model_init.load_state_dict(m3_checkpoint)
convection_model_init = convection_model_init.to(device)

cos_sim, l2_dist = compute_model_similarity(convection_model_init, convection_model)

print(f"Cosine Similarity: {cos_sim}")
print(f"L2 Norm: {l2_dist}")
Initializing Network with Xavier Initialization..
Cosine Similarity: 1.0
L2 Norm: 0.0
ice_model_init = MLP(m4_layers, activation="gelu")
ice_model_init.load_state_dict(m4_checkpoint)
ice_model_init = ice_model_init.to(device)

cos_sim, l2_dist = compute_model_similarity(ice_model_init, ice_model)

print(f"Cosine Similarity: {cos_sim}")
print(f"L2 Norm: {l2_dist}")
Initializing Network with Xavier Initialization..
Cosine Similarity: 1.0
L2 Norm: 0.0
 
 
 
 